# This notebook is intented to develope a ranking system

In [1]:
import pandas as pd
import numpy as np
import re as re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
fill_file="collisions/oms_fills.json"
runs_file='oms_runs.json'
lumi_file='oms_lumisections.json'
hlt_file ='collisions/oms_hltrates.json'


filldf=pd.read_json(fill_file)
runsdf=pd.read_json(runs_file)
lumidf=pd.read_json(lumi_file)
hltdf= pd.read_json(hlt_file)

## Adding place of run in fill

In [3]:
runsdf.columns#[["run_number","duration",'Era']]

Index(['components', 'l1_hlt_mode_stripped', 'l1_key', 'duration', 'b_field',
       'tier0_transfer', 'l1_triggers_counter', 'hlt_physics_throughput',
       'init_lumi', 'last_update', 'beamactive_total_deadtime',
       'delivered_lumi', 'recorded_lumi', 'cmssw_version', 'energy',
       'end_lumi', 'hlt_physics_rate', 'fill_number', 'l1_hlt_mode',
       'end_time', 'trigger_mode', 'l1_key_stripped', 'fill_type_party2',
       'fill_type_party1', 'initial_prescale_index', 'sequence', 'start_time',
       'hlt_physics_size', 'fill_type_runtime', 'hlt_key', 'clock_type',
       'last_lumisection_number', 'l1_rate', 'l1_menu', 'run_number',
       'stable_beam', 'hlt_physics_counter', 'init_lumi_unit',
       'delivered_lumi_unit', 'recorded_lumi_unit', 'end_lumi_unit'],
      dtype='object')

In [4]:
runsdf.set_index("run_number")

,components,l1_hlt_mode_stripped,l1_key,duration,b_field,tier0_transfer,l1_triggers_counter,hlt_physics_throughput,init_lumi,last_update,beamactive_total_deadtime,delivered_lumi,recorded_lumi,cmssw_version,energy,end_lumi,hlt_physics_rate,fill_number,l1_hlt_mode,end_time,trigger_mode,l1_key_stripped,fill_type_party2,fill_type_party1,initial_prescale_index,sequence,start_time,hlt_physics_size,fill_type_runtime,hlt_key,clock_type,last_lumisection_number,l1_rate,l1_menu,stable_beam,hlt_physics_counter,init_lumi_unit,delivered_lumi_unit,recorded_lumi_unit,end_lumi_unit
run_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
315260,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",cosmics2018/v105,l1_trg_cosmics2018/v63,2017,3.801,True,1720492.0,0.013732,0.000000,2018-04-27T01:02:02Z,0.00,0.000000,0.000000,CMSSW_10_1_2_patch2,0.0,0.000000,153.803,6616.0,cosmics2018,2018-04-26 23:50:18+00:00,l1_hlt_cosmics2018/v105,cosmics2018/v63,PROTON,PROTON,0.0,GLOBAL-RUN,2018-04-26 23:16:41+00:00,27.209,COSMICS,/cdaq/cosmic/commissioning2018/CRAFT/v1.2/HLT/V2,LOCAL,85.0,868.345,L1Menu_Collisions2018_0_0_1,False,304742.0,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315262,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",circulating2018/v34,l1_trg_circulating2018/v18,8693,3.801,True,9480160.0,0.003490,0.000000,2018-04-27T03:34:13Z,4.35,0.000000,0.000000,CMSSW_10_1_2_patch2,6499.0,0.000000,50.183,6616.0,circulating2018,2018-04-27 02:25:13+00:00,l1_hlt_circulating2018/v34,circulating2018/v18,PROTON,PROTON,2.0,GLOBAL-RUN,2018-04-27 00:00:20+00:00,29.778,PROTONS,/cdaq/special/Circulating2018/v1.0.3/HLT/V2,LHC,371.0,1111.201,L1Menu_Collisions2018_0_0_1,False,428142.0,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315264,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v74,l1_trg_collisions2018/v10,6119,3.801,True,295208405.0,1.075522,0.842940,2018-04-27T05:19:53Z,7.92,42.924551,39.385570,CMSSW_10_1_2_patch2,6499.0,0.725879,1055.363,6616.0,collisions2018,2018-04-27 04:14:41+00:00,l1_hlt_collisions2018/v74,collisions2018/v10,PROTON,PROTON,3.0,GLOBAL-RUN,2018-04-27 02:32:42+00:00,6493.351,PROTONS,/cdaq/physics/Run2018/2e34/v1.1.0/HLT/V5,LHC,261.0,48897.560,L1Menu_Collisions2018_0_0_1,True,6371642.0,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315265,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v74,l1_trg_collisions2018/v10,1389,3.801,True,44393104.0,0.987777,0.725641,2018-04-27T05:44:08Z,36.57,10.303512,6.651641,CMSSW_10_1_2_patch2,6499.0,0.699951,1015.209,6616.0,collisions2018,2018-04-27 04:41:09+00:00,l1_hlt_collisions2018/v74,collisions2018/v10,PROTON,PROTON,3.0,GLOBAL-RUN,2018-04-27 04:18:00+00:00,990.096,PROTONS,/cdaq/physics/Run2018/2e34/v1.1.0/HLT/V5,LHC,58.0,44289.950,L1Menu_Collisions2018_0_0_1,True,1017592.0,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315266,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v74,l1_trg_collisions2018/v10,819,3.801,True,0.0,NaN,0.700102,2018-04-27T06:00:10Z,100.00,6.610566,0.000000,CMSSW_10_1_2_patch2,6499.0,0.685267,NaN,6616.0,collisions2018,2018-04-27 04:55:46+00:00,l1_hlt_collisions2018/v74,collisions2018/v10,PROTON,PROTON,3.0,GLOBAL-RUN,2018-04-27 04:42:07+00:00,NaN,PROTONS,/cdaq/physics/Run2018/2e34/v1.1.0/HLT/V5,LHC,33.0,NaN,L1Menu_Collisions2018_0_0_1,True,NaN,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315267,"[CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...",collisions2018/v74,l1_trg_collisions2018/v10,5716,3.801,True,232524561.0,0.865909,0.685031,2018-04-27T07:38:12Z,6.29,38.581152,36.354441,CMSSW_10_1_2_patch2,6499.0,0.616229,953.282,6616.0,collisions2018,2018-04-27 06:33:45+00:00,l1_hlt_collisions2018/v74,collisions2018/v10,PROTON,PROTON,3.0,GLOBAL-RUN,2018-04-27 04:58:29+00:00,4925.064,PROTONS,/cdaq/physics/Run2018/2e34/v1.1.0/HLT/V5,LHC,244.0,40882.477,L1Menu_Collisions2018_0_0_1,True,5422021.0,10^{34}cm^{-2}s^{-1},pb^{-1},pb^{-1},10^{34}cm^{-2}s^{-1}
315268,"[CSC, CTPPS, CTPPS_

First filter out the runs that DO NOT belong to any fill

In [12]:
runs_slimdf=runsdf[['components', 'init_lumi',  'last_update', 'delivered_lumi', 'recorded_lumi','end_lumi',
                    'fill_number','last_lumisection_number',
                    'l1_rate', 'run_number',
       'stable_beam', 'hlt_physics_counter']]

In [13]:
def has_fill(df):
    dfcopy=df[df.fill_number.isna() == False].copy()
    return dfcopy

def add_loc_wrt_fill(df):
    import numpy as np
    dfcopy= df.copy()
    dfcopy["Fill location"] = np.empty(len(dfcopy))
    for i in range(len(dfcopy.index)):
        dfcopy['Fill location'].iloc[i]="({}/{})".format(i+1,len(dfcopy.index))    
    return dfcopy

In [14]:
runswithfill=has_fill(runs_slimdf)
runswithfill[['run_number','fill_number']]

,run_number,fill_number
0,315260,6616.0
1,315262,6616.0
2,315264,6616.0
3,315265,6616.0
4,315266,6616.0
5,315267,6616.0
6,315268,6616.0
8,315270,6616.0
9,315273,6616.0


In [15]:
runsdf_filLoc=add_loc_wrt_fill(runswithfill)

/cvmfs/sft.cern.ch/lcg/views/LCG_100/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Now we have a location of the run with respect to the fill number.

In [16]:
runsdf_filLoc[["run_number","fill_number", 'Fill location']]

,run_number,fill_number,Fill location
0,315260,6616.0,(1/9)
1,315262,6616.0,(2/9)
2,315264,6616.0,(3/9)
3,315265,6616.0,(4/9)
4,315266,6616.0,(5/9)
5,315267,6616.0,(6/9)
6,315268,6616.0,(7/9)
8,315270,6616.0,(8/9)
9,315273,6616.0,(9/9)


In [17]:
runsdf_filLoc.set_index(['fill_number','run_number'])

components  \
fill_number run_number                                                      
6616.0      315260      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315262      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315264      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315265      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315266      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315267      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315268      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315270      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   
            315273      [CSC, CTPPS, CTPPS_TOT, DAQ, DCS, DQM, DT, ECA...   

                        init_lumi           last_update  delivered_lumi  \
fill_number run_number                                                    
6616.0      315260       0.000000  2018-04-27T01:02:02Z        0.000000   
            315262       0.000000  2018-04-27T03:34:13Z        0.000000   
            315264       0.842940  2018-04-27T05:19:53Z       42.924551   
            315265       0.725641  2018-04-27T05:44:08Z       10.303512   
            315266       0.700102  2018-04-27T06:00:10Z        6.610566   
            315267       0.685031  2018-04-27T07:38:12Z       38.581152   
            315268       0.615901  2018-04-27T07:44:56Z        2.430633   
            315270       0.611140  2018-04-27T07:53:08Z       80.010320   
            315273       0.000000  2018-04-27T12:37:16Z        0.000000   

                        recorded_lumi  end_lumi  last_lumisection_number  \
fill_number run_number                                                     
6616.0      315260           0.000000  0.000000                     85.0   
            315262           0.000000  0.000000                    371.0   
            315264          39.385570  0.725879                    261.0   
            315265           6.651641  0.699951                     58.0   
            315266           0.000000  0.685267                     33.0   
            315267          36.354441  0.616229                    244.0   
            315268           0.000000  0.611543                      3.0   
            315270          78.149695  0.476770                    655.0   
            315273           0.000000  0.000000                     47.0   

                          l1_rate  stable_beam  hlt_physics_counter  \
fill_number run_number                                                
6616.0      315260        868.345        False             304742.0   
            315262       1111.201        False             428142.0   
            315264      48897.560         True            6371642.0   
            315265      44289.950         True            1017592.0   
            315266            NaN         True                  NaN   
            315267      40882.477         True            5422021.0   
            315268          4.161         True                  NaN   
            315270      33561.880         True           11451299.0   
            315273      30617.836         True             359784.0   

                       Fill location  
fill_number run_number                
6616.0      315260             (1/9)  
            315262             (2/9)  
            315264             (3/9)  
            315265             (4/9)  
            315266             (5/9)  
            315267             (6/9)  
            315268             (7/9)  
            315270             (8/9)  
            315273             (9/9)

## Working with the Lumi dataframe

In [11]:
lumidf.set_index(['run_number','lumisection_number'])

rp_time_ready  cscp_ready  physics_flag  \
run_number lumisection_number                                            
315265     1                             NaN        True         False   
           2                             NaN        True         False   
           3                             NaN        True         False   
           4                             NaN        True          True   
           5                             NaN        True          True   
           6                             NaN        True          True   
           7                             NaN        True          True   
           8                             NaN        True          True   
           9                             NaN        True          True   
           10                            NaN        True          True   
           11                            NaN        True          True   
           12                            NaN        True          True   
           13                            NaN        True          True   
           14                            NaN        True          True   
           15                            NaN        True          True   
           16                            NaN        True          True   
           17                            NaN        True          True   
           18                            NaN        True          True   
           19                            NaN        True          True   
           20                            NaN        True          True   
           21                            NaN        True          True   
           22                            NaN        True          True   
           23                            NaN        True          True   
           24                            NaN        True          True   
           25                            NaN        True          True   
           26                            NaN        True          True   
           27                            NaN        True          True   
           28                            NaN        True          True   
           29                            NaN        True          True   
           30                            NaN        True          True   
           31                            NaN        True          True   
           32                            NaN        True          True   
           33                            NaN        True          True   
           34                            NaN        True          True   
           35                            NaN        True          True   
           36                            NaN        True          True   
           37                            NaN        True          True   
           38                            NaN        True          True   
           39                            NaN        True          True   
           40                            NaN        True          True   
           41                            NaN        True          True   
           42                            NaN        True          True   
           43                            NaN        True          True   
           44                            NaN        True          True   
           45                            NaN        True          True   
           46                            NaN        True          True   
           47                            NaN        True          True   
           48                            NaN        True          True   
           49                            NaN        True          True   
           50                            NaN        True          True   
           51                            NaN        True          True   
           52                            NaN        True          True   
           53                      